In [13]:
import numpy as np 
import pandas as pd 
from tensorflow.keras.layers import Dense,Flatten, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential
import cv2
from tensorflow.keras.applications.vgg19 import VGG19
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import random

In [14]:
df = pd.read_csv("/Users/yeseulseo/Section4/project/archive/full_df.csv")
df.head()


,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [15]:
#특수문자제거
df['labels']= df['labels'].str.replace(pat=r'[^\w]',repl=r'',regex=True)
df.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,N,"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,N,"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,D,"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,D,"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,D,"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [16]:
df["labels"].value_counts()

N    2873
D    1608
O     708
C     293
G     284
A     266
M     232
H     128
Name: labels, dtype: int64

## EDA

In [17]:
#눈질병이외의 질병 데이터는 삭제
condition = (df['labels']=='D') | (df['labels']=='H') | (df['labels']=='O')| (df['labels']=='M')
for i in df[condition].index:
    df =  df.drop(i)

df = df.reset_index(drop=True) #index 초기화
df["labels"].value_counts()    

N    2873
C     293
G     284
A     266
Name: labels, dtype: int64

In [18]:
#타겟을 int로 수정 , 문자로 하니깐 안됐음
df1= df.replace({'labels': {'N': 0,'C': 1,'G': 2,'A': 3}})

df1.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,0,"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,0,"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,8,59,Male,8_left.jpg,8_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,0,"[1, 0, 0, 0, 0, 0, 0, 0]",8_right.jpg
3,10,70,Male,10_left.jpg,10_right.jpg,epiretinal membrane,normal fundus,0,0,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,0,"[1, 0, 0, 0, 0, 0, 0, 0]",10_right.jpg
4,24,75,Female,24_left.jpg,24_right.jpg,normal fundus,cataract,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,1,"[0, 0, 0, 1, 0, 0, 0, 0]",24_right.jpg


In [20]:
df2=df1[['ID','Patient Age'	,'Patient Sex','filepath',	'labels', 'filename']]
df2.head()

,ID,Patient Age,Patient Sex,filepath,labels,filename
0,0,69,Female,../input/ocular-disease-recognition-odir5k/ODI...,0,0_right.jpg
1,1,57,Male,../input/ocular-disease-recognition-odir5k/ODI...,0,1_right.jpg
2,8,59,Male,../input/ocular-disease-recognition-odir5k/ODI...,0,8_right.jpg
3,10,70,Male,../input/ocular-disease-recognition-odir5k/ODI...,0,10_right.jpg
4,24,75,Female,../input/ocular-disease-recognition-odir5k/ODI...,1,24_right.jpg


In [21]:
#사진 경로 앞부분은 나랑 다르니깐 삭제

for i in range(len(df)) :
  df2['filepath'].loc[i]= df2['filepath'].loc[i][43:]


df2.head()

/var/folders/yp/r9q2z09j2rj4vfhr9q8r9vl40000gn/T/ipykernel_11192/3427607062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['filepath'].loc[i]= df2['filepath'].loc[i][43:]


,ID,Patient Age,Patient Sex,filepath,labels,filename
0,0,69,Female,ODIR-5K/Training Images/0_right.jpg,0,0_right.jpg
1,1,57,Male,ODIR-5K/Training Images/1_right.jpg,0,1_right.jpg
2,8,59,Male,ODIR-5K/Training Images/8_right.jpg,0,8_right.jpg
3,10,70,Male,ODIR-5K/Training Images/10_right.jpg,0,10_right.jpg
4,24,75,Female,ODIR-5K/Training Images/24_right.jpg,1,24_right.jpg


In [22]:
#내 사진 경로
path='/Users/yeseulseo/Section4/project/archive/ODIR-5K/'

In [23]:
#사진불러오기 방법2
im = cv2.imread(path+df2['filepath'][0], cv2.IMREAD_COLOR)
print (type(im))

<class 'numpy.ndarray'>


In [24]:
#리스트를 만들어서 사진 넣기 
dataset = []

for i in tqdm(range(len(df2))):
  img = cv2.imread(path+df2['filepath'][i],cv2.IMREAD_COLOR) #사진 불러오기
  img = cv2.resize(img, (224, 224)) #사진크기 조절
  dataset.append([np.array(img), np.array(df2['labels'][i])])

dataset

100%|██████████| 3716/3716 [01:25<00:00, 43.36it/s]


[[array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         ...,
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]],
  
         [[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]]], dtype=uint8),
  array(0)],
 [array([[[0, 0, 0],
          [0, 0, 0],
          [0, 0, 0],
          ...,
          

In [34]:
x = np.array([i[0] for i in dataset]).reshape(-1,224,224,3)
x = x.astype('float32')/255 #정규화
y = np.array([i[1] for i in dataset])

## 모델링


In [35]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [36]:
x_train.shape

(2972, 224, 224, 3)

In [37]:
model3 = Sequential()
model3.add(ResNet50(weights="imagenet",include_top = True,input_shape=(224,224,3)))
model3.add(Flatten())
model3.add(BatchNormalization())
model3.add(Dropout(0.2))
model3.add(Dense(4,activation="softmax")) #타겟 4개니깐 

# ResNet-50 model is already trained, should not be trained
model3.layers[0].trainable = True #예측을 자꾸 똑같이 나왔는데 이코드나와서 해결

In [38]:
model3.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  #차원이 뭔가 달라서 categorical_crossentropy는 안됨
    metrics=['acc']
)

In [39]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1000)              25636712  
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 batch_normalization (BatchN  (None, 1000)             4000      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense_1 (Dense)             (None, 4)                 4004      
                                                                 
Total params: 25,644,716
Trainable params: 25,589,596
Non-trainable params: 55,120
_____________________________________

In [40]:
earlystop = EarlyStopping(monitor="val_acc",patience=3,verbose=1) 
#mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max',patience=10, verbose=1, save_best_only=True)

In [41]:
history = model3.fit(x_train,y_train,batch_size=64,epochs=20,validation_data=(x_test,y_test),
                    verbose=1,callbacks=[earlystop])

Epoch 1/20


2022-07-27 08:37:44.564174: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - ETA: 0s - loss: 0.9203 - acc: 0.7106

2022-07-27 08:39:47.105410: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


47/47 [==============================] - 135s 3s/step - loss: 0.9203 - acc: 0.7106 - val_loss: 0.7545 - val_acc: 0.7890
Epoch 2/20
47/47 [==============================] - 157s 3s/step - loss: 0.6967 - acc: 0.7732 - val_loss: 0.7695 - val_acc: 0.7890
Epoch 3/20
47/47 [==============================] - 203s 4s/step - loss: 0.6269 - acc: 0.7985 - val_loss: 0.7908 - val_acc: 0.7890
Epoch 4/20
47/47 [==============================] - 241s 5s/step - loss: 0.5892 - acc: 0.8106 - val_loss: 0.7696 - val_acc: 0.7890
Epoch 4: early stopping


In [ ]:
#model1 = load_model('best_model2.h5')

In [42]:
loss,accuracy = model3.evaluate(x_test,y_test)
print("loss:",loss)
print("Accuracy:",accuracy)

24/24 [==============================] - 20s 821ms/step - loss: 0.7696 - acc: 0.7890
loss: 0.7695610523223877
Accuracy: 0.7889785170555115


In [48]:
im_test1= cv2.imread('/Users/yeseulseo/Section4/project/ocular_new/archive/dataset/1_normal/NL_001.png')
im_test1 = cv2.cvtColor(im_test1, cv2.COLOR_BGR2RGB)
im_test1 = cv2.resize(im_test1, (224,224))
im_test1 = np.expand_dims(im_test1, axis=0) #사진 차원이 달라서 조정
y_pred_test1=model3.predict(im_test1)
pred1 = y_pred_test1[0].argmax()
print(pred1)

1/1 [==============================] - 0s 32ms/step
0
